# Buchhandelsdaten in Vufind als Grundlage für PDA (Patron Driven Aquisition) am MPIfG / Version 2: Updates laden statt Vollimport

Einbindung von freundlicherweise von Schweitzer Fachinformation zur Verfügung gestellten Daten.   
(Auswahl über passend konfigurierte Neuerscheinungsabfragen in unserem Kundenprofil).   

#### Gründe für die Umstellung auf Updates: Durch Einspielen der Daten in Vufind ergibt sich eine zeitliche Diskrepanz, in der Titel bereits in Aleph gelöscht sind, aber in Vufind noch bestellbar. Zudem werden die Aleph-IDs hochgezähl, denn bei jedem Einspielen werden neue Nummern vergeben. Durch die Updates bleiben die Titel weiterhin verfügbar und das Hochzählen der IDs hält sich im Rahmen. Das Skript beschleunigt sich durch Reduzierung der Abfragen am Server.
*Kleiner Nachteil: der manuelle Aufwand vergrößert sich. Mehrere Dateien müssen in Aleph eingespielt und verarbeitet werden, aber hält sich nach wie vor im vertretbaren Rahmen.*

Das Jupyter Notebook arbeitet mit Python 3.8.10 und wurde mit Visual Studio Code 1.62.3 erstellt 


#### Arbeitsschritte im Code:

> Vorarbeiten:   
  - Notwendige Pandas Libraries aufrufen
  - Serverprüfung auf funktionierende Verbindung zum Aleph-X-Server    


> Daten abholen und einlesen:   
  1. Buchhandelsdaten von Schweitzer 
  2. Aleph-Konkordanz Aleph-ID /Schweitzer ID  (erzeugt tagesaktuell per p-print-03 in Aleph)
     - Aufbereiten der Daten: Schweitzer ID extrahieren und Aleph-ID mit Nullen auffüllen
  3. Daten zusammenführen in einem df    


> Daten aufbereiten:   
  1. Buchhandelsdaten prüfen und vorbereiten
     1. Identifizierung von Titeln in Aleph, die nicht mehr im Datensatz sind und schreiben in Datei "ges02_loeschen_1"
     2. Dublettencheck innerhalb der Buchhandelsdaten
     3. Trennung der Daten in "in Aleph vorhanden" und "neu"  
  2. Bestandsabfragen:
     1. Ganz neue Titel   
        - Bestandsabgleich durch Abfragen (GES und EBX) auf dem Aleph-Server   
     2. In Aleph vorhanden
        - Bestandsabgleich durch Abfrage EBX auf dem Aleph-Server  
     3. Exportvorbereitungen:
        1. Neue Titel
        2. In Aleph vorhandene Titel   
            Identifizierung von zu löschenden Titeln "ges02_loeschen_2" und der zu aktualisierenden
  3. Exportdateien aufbereiten:
     1. Aufbereiten neuer Titel
     2. Aufbereiten der vorhandenen Titel   


> Informationssammlung
   1. Log-Datei mit Rahmendaten wird fortlaufend geschrieben
   2. Ausgabe bestimmter Titelgruppen als csv-Datei  


<hr>

## Vorarbeiten

### Pandas Libraries laden

In [1]:
import pandas as pd                                                # für das Arbeiten mit der CSV-Datei
import urllib.request                                              # für das Abrufen der URL
import requests                                                    # für die Bestandsabfragen 
pd.options.mode.chained_assignment = None                          # default='warn' abschalten beim Beschreiben der neuen Spalten
import time                                                        # für das Schreiben des Datums Logdatei und Excel-Export und Arbeiten mit dem Erscheinungsdatum
import datetime                                                    # für das Berechnen des Updates
import numpy as np                                                 # für das Bearbeiten von Spalten


### Prüfung, ob die Verbindung zum Aleph-Server für Abfragen korrekt funktioniert:

    Nur zugelassene IPs können diese Schnittstelle abfragen.

In [2]:
test= "http://aleph.mpg.de/X?op=find&base=ges01&request=IBS=9783482648434"

reply = requests.get(test).text
a = reply.find("Forbidden")
b =  reply.find("?xml")

if (a > 50):
    print("Es gibt ein Problem mit dem Server")
if (b == 1):
    print("Der Server antwortet korrekt")

Der Server antwortet korrekt


In [3]:
test= "http://aleph.mpg.de/X?op=find&base=ges01&request=IBS=9783865058041"

reply = requests.get(test).text 

print(reply) 

<?xml version = "1.0" encoding = "UTF-8"?>
<find>
<set_number>576142</set_number>
<no_records>000000001</no_records>
<no_entries>000000001</no_entries>
<session-id>E5CE11Y6HSTXVBB8M7YIIN6G27FTSLRJEYXCXMLMPE7YKGACUK</session-id>
</find>
 


<hr>


## Datensätze abholen und einlesen

# Erste Schritte manuell durchführen, um enthaltene Prüfroutinen im Blick zu behalten

<hr>

### 1. Datensätze von Schweitzer

*einlesen in df*

In [3]:
url = "https://content.schweitzer-online.de/static/content/export/mpifg/export.csv"  # Abruf, der von Schweitzer zur Verfügung gestellten Daten
checkout_file = "./input/export.csv"  
urllib.request.urlretrieve(url, checkout_file)

('./input/export.csv', <http.client.HTTPMessage at 0x1c46c106e80>)

In [4]:
df = pd.read_csv('./input/export.csv', encoding = 'UTF-8', sep=';' , keep_default_na=False) # muss encoding angeben und Trennzeichen, NaN (= leere Werte) direkt beim Import entfernen, da sie später Probleme machen

In [5]:
#Aufgrund eines zusätzlichen Zeilenumbruchs im Datensatz gab es im Juni 2022 ein Problem, darum kleine Prüfroutine eingebaut ob in der Import-Datei die Anzahl der Datensätze mit der Anzahl der object-Ids übereinstimmt
g = df.shape[0]
h= df['object_id'].count()

print("Zeilen in Datei:", g)
print("Object_IDs:     ", h)


Zeilen in Datei: 23894
Object_IDs:      23894


# Ab dem nächsten Schritt kann Skript durchlaufen, Prüfroutinen sind erfolgt

<hr>

### 2. Aleph-Konkordanz Aleph-ID / Schweitzer ID

In [6]:
now = time.strftime('%Y%m%d')  # %H:%M:%S
input_file = './input/ids'+now      #input_file wird tagesaktuell aus Aleph gezogen und auf diesem Wege mit der entsprechenden Endung eingelesen

In [7]:

df_alephIDs = pd.read_fwf(input_file, encoding = 'UTF-8', sep='\s\s', header=None, keep_default_na=False) 
df_alephIDs.columns=["ID","Field","L","Content"]
df_alephIDs

#Spalte 0 = Aleph-IDs müsste vor Export mit Nullen aufgefüllt werden 
#Spalte 3 = object_id aus df, hier muss zum Abgleich $$a entfernt werden


,ID,Field,L,Content
0,62432,20,L,$$a160009340
1,62433,20,L,$$a158945668
2,62435,20,L,$$a159450684
3,62436,20,L,$$a156899389
4,62437,20,L,$$a160055285
...,...,...,...,...
20646,94120,20,L,$$a164829102
20647,94121,20,L,$$a163840881
20648,94122,20,L,$$a164644335
20649,94123,20,L,$$a160226864


*Datenfelder aufbereiten: Schweitzer-ID extrahieren und Aleph-ID ins richtige Format bringen*

In [8]:
df_alephIDs["object_id"] = df_alephIDs["Content"].astype(str).str.slice(start=3,stop=13).apply(int)   #Spalte mit object-Ids herausschneiden und wieder zur Zahl definieren

df_alephIDs["object_id"]

0        160009340
1        158945668
2        159450684
3        156899389
4        160055285
           ...    
20646    164829102
20647    163840881
20648    164644335
20649    160226864
20650    162283229
Name: object_id, Length: 20651, dtype: int64

In [9]:
#Aleph-ID-Spalte mit Nullen Auffüllen
df_alephIDs["ids"] = df_alephIDs['ID'].apply(lambda x: f"{x:09d}")
df_alephIDs["ids"]

0        000062432
1        000062433
2        000062435
3        000062436
4        000062437
           ...    
20646    000094120
20647    000094121
20648    000094122
20649    000094123
20650    000094124
Name: ids, Length: 20651, dtype: object

In [10]:
#nach Dubletten vom letzten Mal vorsichtshalber, check nach doppelten object-ids
df_alephIDs["object_id"].value_counts()

162793471    1
162707260    1
163220068    1
163832942    1
131948949    1
            ..
160885494    1
164005750    1
162501848    1
160892122    1
156993537    1
Name: object_id, Length: 20651, dtype: int64

### 3. Daten zu einem Frame zusammenführen

In [11]:
# Zusammenführung erfolgt einmal mit concat - erlaubt die nicht mehr verwendeten Aleph-IDs zu identifizieren 
# und einmal mit join, dann erhalte ich nur die Titel, die auch im neuen Datensatz sind zur weiteren Verarbeitung!
df_oi= df.set_index("object_id")                          #object-ID zum Index für beide Datenframes
df_aleph_oi = df_alephIDs.set_index("object_id")
df_update_aleph = pd.concat([df_oi, df_aleph_oi], axis=1)

df_update_join = df_oi.join(df_aleph_oi)                    



<hr>

### LOG-Datei für den Prozess, zur Dokumentation des Imports und als Kontrollanzeige hier


In [12]:
x = df.shape[0]
print('Anzahl der enthaltenen Datensätze:', x)
#print('vorhandene ISBNs:', df_update["isbn_ean"].shape[0])

z = df_alephIDs["ids"].count()
y = df_update_join["ids"].count()

print('-------------------------')
print('Aleph-IDs anfangs', z)
print('Aleph-IDs nach join' , y)

timestr = time.strftime('%d.%m.%Y - %H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('Logdatei PDA-Import vom ')
    log.write(timestr)
    log.write('\n------------------------------------------\n')
    log.write('Gelieferte Datensätze:             ' + str(x))

Anzahl der enthaltenen Datensätze: 23894
-------------------------
Aleph-IDs anfangs 20651
Aleph-IDs nach join 20594


<hr>

# Problemlösung der Aleph-Dubletten


df_alephIDs["object_id"].value_counts

df_aleph_einzel = df_alephIDs.drop_duplicates("object_id", keep=False)  #Auslesen der Eintraege mit einzelner object_id

df_aleph_doppelt = df_alephIDs.groupby("object_id").filter(lambda g: (g.nunique() >1).any()) #Doppelte object_ids rausziehen
df_aleph_single = df_aleph_doppelt.sort_values(by=["object_id", "ids"], ascending =False).drop_duplicates(subset=["object_id"], keep='first')  #von Dubletten 1. behalten 

df_aleph_vorhanden = df_aleph_einzel.append(df_aleph_single)

<hr>

## Daten aufbereiten

<hr>

### 1. Identifizierung von Titeln in Aleph-Dubletten, die nicht mehr im Datensatz sind

In [13]:
df_update_aleph.replace({np.nan: None})

,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,media_type,...,cover,predecessor,follower,last_modified,watchlist_name,ID,Field,L,Content,ids
object_id,,,,,,,,,,,,,,,,,,,,,
218910,9.78341e+12,Grundkurs Steuerrecht,,Rainer Wernsmann;Christian Thiemann,C.H. Beck,Grundkurse,,1,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2022-11-25 19:49:41,brombeere Politik #69.1,88988,20,L,$$a218910,000088988
237277,9.78383e+12,Käuferverhalten,Eine marketingorientierte Einführung,Alfred Kuß;Torsten Tomczak;Silke Lennerts,UTB,Grundwissen der Ökonomik Band 1604,,5,"5., komplett überarbeitete Auflage 2017",hardcover,...,https://content.schweitzer-online.de/static/ca...,9783825216047,,2022-09-18 19:05:17,Himbeere Wirtschaft #26,68192,20,L,$$a237277,000068192
259076,9.78387e+12,Geschlossene Gesellschaft,Ein Reichtumsbericht,Dennis Gastmann,Rowohlt Berlin,,,,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2021-06-30 20:57:28,,None,None,None,None,None
1681954,9.789e+12,The Workers' Opposition in the Russian Communi...,"Documents, 1919-30",,Brill,Historical Materialism Book Series 236,,,,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2022-12-08 18:57:03,diverse1,62977,20,L,$$a1681954,000062977
1958724,9.78042e+12,"Population, Development and Welfare in the His...",,Claudia Sunna,Routledge,Routledge Studies in the History of Economics,,1,,hardcover,...,,,,2022-11-20 17:29:10,brombeere Politik #43,72194,20,L,$$a1958724,000072194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166320271,9.78396e+12,Signifikant 4/5 - 2021/2022,,Rainer E. Zimmermann;Wolfgang Hofkirchner;Ralp...,wvb Wissenschaftlicher Verlag Berlin,Signifikant 4,,1,Erstausgabe,hardcover,...,https://content.schweitzer-online.de/static/ca...,,,2022-12-15 05:48:21,brombeere Politik #84,None,None,None,None,None
166336905,9.78288e+12,Robert Poujade et le ministère de l'environnement,,,Peter Lang Verlag,Georges Pompidou - Études 12,,,,hardcover,...,,,,2022-12-16 05:33:53,brombeere Politik #84,None,None,None,None,None
166337167,9.78363e+12,Mujeres y discursos de paz en la historia,,,Peter Lang Verlag,,,,,hardcover,...,,,,2022-12-18 20:06:31,Himbeere Wirtschaft #83,None,None,None,None,None


In [14]:
df_aleph_loeschen = df_update_aleph[df_update_aleph['isbn_ean'].isna()]  


In [15]:
df_aleph_loeschen

,isbn_ean,title,subtitle,contributor,publisher,series,thesis,edition_number,edition_text,media_type,...,cover,predecessor,follower,last_modified,watchlist_name,ID,Field,L,Content,ids
object_id,,,,,,,,,,,,,,,,,,,,,
2543676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,84226.0,20.0,L,$$a2543676,000084226
2544471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,84233.0,20.0,L,$$a2544471,000084233
2571635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,86866.0,20.0,L,$$a2571635,000086866
2578273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,88955.0,20.0,L,$$a2578273,000088955
11907234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,92083.0,20.0,L,$$a11907234,000092083
133861100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,71981.0,20.0,L,$$a133861100,000071981
159278779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,64703.0,20.0,L,$$a159278779,000064703
159495076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,82400.0,20.0,L,$$a159495076,000082400
159872342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,63894.0,20.0,L,$$a159872342,000063894


In [16]:
# Schreiben der Loeschen-datei!

with open("./output/ges02_weg", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_loeschen.index:
        fa.write(df_aleph_loeschen["ids"][i]+'GES02'+'\n')

<hr>

## 2. Dublettencheck innerhalb gelieferten Buchhandelsdaten 

Aufgaben im Rahmen des Dublettencheck:
1. Dublettenkontrolle anhand von Titel, Untertitel und Autor 
   - Zunächst Behebung der unsauberen Titel / Untertitel-Trennung für korrekteren Abgleich
   - Trennung der Datensätze in Dubletten und "Einzeltitel"
     - Einzeltitel werden direkt für Bestandsprüfung vorgemerkt
     - Dubletten werden auf neueste Version reduziert und diese der Bestandsprüfungsdatei hinzugefügt

### Die Dublettenkontrolle Anhand von Titel, Untertitel und Autor

*Entfernen von Untertiteln aus der Titelspalte, Extrahieren von Untertiteln und Abgleich mit Untertitelspalte und Schreiben der vorhandenen Informationen in neue Untertitel-Spalte.   
Durch diese Spalte werden ca. 1/3 mehr Dubletten erkannt, als ohne die Bereinigung. *

In [17]:
df = df_update_join[df_update_join['isbn_ean'].notna()]  #jetzt alle die zu verarbeiten sind rausziehen
df.reset_index(inplace=True) 

In [18]:
neu = df["title"].str.split(':', n = 1, expand = True)  #Titel am 1. Doppelpunkt splitten und getrennt in neue Felder schreiben
df["title_sep"]= neu[0]
df["subtitle_sep"]= neu[1]

df["subtitle_sep"] = df["subtitle_sep"].replace(np.nan, '', regex=True) #NaN-Werte stören, darum raus damit ...

In [19]:
comparison = np.where(df["subtitle"] == df["subtitle_sep"], '', df["subtitle"])    # Abgleich - wenn in beiden das Gleiche steht, dann ursprüngliches "Subtitle"-Feld nehmen
df["subtitle_comparison"] = comparison  

comparison2 = np.where(df["subtitle"] < df["subtitle_sep"], df["subtitle_sep"], '') # Wenn nur in "subtitle_sep" Infos stehen, diese übernehmen, das ist noch nicht ganz sauber, da hier manchmal anderes steht als in "subtitle"
df["subtitle_comparison2"] = comparison2 

df["subtitle_all"] = df["subtitle_comparison"]+df["subtitle_comparison2"]          # Beide Informationen in neuer Subtitle-Spalte zusammenführen

In [20]:
df["short_title"] = df["title_sep"] + ' ' + df["subtitle_all"] + ' / ' + df["contributor"]  # aus den bereinigten Daten einen Kurztitel erzeugen, der dann für den Dublettencheck verwendet wird

In [21]:
df_dubletten = df.groupby("short_title").filter(lambda g: (g.nunique() >1).any()) # schreibt alle mehrfach vorhandenen Titel in ein eigenes Datenframe

In [22]:

df_dubl_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep='first')   # sortiert Dubletten nach Jahr und schreibt den jeweils ersten (= neuesten) Eintrag in neues Dataframe


In [23]:
df_ohne_dubletten = df.drop_duplicates("short_title", keep=False)       #durch ""keep=False" werden alle nicht-Dubletten rausgezogen

In [24]:
df_dubl_nicht_einspielen = df_dubletten.sort_values(by=["short_title", "publication_year"], ascending =False).drop_duplicates(subset=["short_title"], keep=False)

#Kontrolle, ob es in Dubletten Titel gibt, die schon in Aleph sind
df_dubl_nicht_einspielen["ids"].count()


0

In [25]:
df_einspielen = df_ohne_dubletten.append(df_dubl_einspielen)                    # die ausgewählten Dubletten und alle Nicht-Dubletten werden in ein Datenframe zusammengeführt
df_einspielen.reset_index(inplace=True)                                         # für weitere Bearbeitung index-reset nötig

<hr>

## 3. Überblick zu den Daten und Trennen in "in Aleph vorhanden" und "neu"

In [26]:
m = df_einspielen.shape[0]
n = df_einspielen["ids"].count()
print("Zum Einspielen:", m)
print("Davon in Aleph:", n)
print("Neue Titel:", m-n)

Zum Einspielen: 20678
Davon in Aleph: 18416
Neue Titel: 2262


In [27]:
df_in_aleph = df_einspielen.dropna(subset=['ids'])
df_in_aleph.reset_index(inplace=True)
df_in_aleph.shape[0]

# zur Prüfung am GES und Ebx Bestand geprüft werden

18416

In [28]:
df_ganz_neu = df_einspielen[df_einspielen['ids'].isnull()]

df_ganz_neu.reset_index(inplace=True)
df_ganz_neu.shape[0]

# diese müssen ebenfalls am Ges und Ebx Bestand geprüft werden, durch Trennung der Sets schnellere Bearbeitung

2262

<hr>

## 2. Bestandsabfragen



*URLs für die Abfrage über den X-Server unseres Bibliothekssystems werden erzeugt und über die ISBN eine Abfrage auf Bestand gemacht. Die Abfrage funktioniert nur für zugelassene IPs (darum oben die Prüfung).  
Für die Abfrage in unseren Bestand ist die ISBN sehr gut, da in den Titeldaten alle im Buch befindlichen ISBNs - auch die anderer Ausgabeformen - mit übernommen sind. Beim MPG-Ebooks Katalog handelt sich um Daten von Verlagen, die sich in ihrer Qualität und Informationsumfang sehr unterscheiden. Hier wird noch zu prüfen sein, inwieweit ein anderer Abfragemechansimus gewählt werden muss.*  

### 1. Ganz neue Titel  

= df_ganz_neu


In [29]:
df_ganz_neu["url_ges"] = df_ganz_neu["isbn_ean"].apply(lambda x: f"ges_link{x}".replace('ges_link','http://aleph.mpg.de/X?op=find&base=ges01&request=IBS='))

#Es funktionierte nicht, dass die URLs an die vorhandenen ISBNs einfach so angefügt werden, darum der Workaround mit einem Platzhalter, der sich dann über replace vom richtigen Link überschreiben ließ.

#### URLs für Ebooks-Katalog erzeugen 

In [30]:
df_ganz_neu["url_ebx"] = df_ganz_neu["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))


#### Abfragen beim Server

##### Zunächst für die Daten des MPIfG 

*Vorgehensweise: Abfrage und Sammeln der Antworten in einer Datei, diese Antworten werden dann in Ausdrücke "übersetzt" - "vorhanden" und "neu" und diese Daten in eine Spalte ins Dataframe zur weiteren Auswertung übertragen.   
Schwierigkeit hier war, die Sammlung der Antworten zu den einzelnen Titeln, um sie in das Datenframe einzuspielen. Der störende XML-Header der Antworten wird erst gar nicht in die Datei geschrieben.   *   

In [31]:
with open('./input/server_responses', 'w') as fn:  
    for url in df_ganz_neu["url_ges"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

In [32]:


with open('./input/server_responses', 'r') as f:
    with open('./input/server_responses_transfered', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf = pd.read_fwf('./input/server_responses_transfered', names=["Abfrage_ges"])
df_result = df_ganz_neu.join(df_fwf)                    #df_result = pd.concat([df_ganz_neu, df_fwf], axis=1), funktioniert nicht mehr                  

    Randnotiz: 
    Bei 2400 Titels brauchte der Abgleich ca 350 Sekunden

In [33]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind, wird mittelfristig rausfallen
x = df_ganz_neu.shape[0]
y = df_fwf.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 2262 
Anzahl der Antworten vom Server:    2262


##### Datenabgleich mit dem Bestand des MPG Ebooks-Katalog

*Vorgehensweise analog Bestandsabfrage MPI.*

In [34]:
with open('./input/server_responses_ebx', 'w') as fn:  
    for url in df_result["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx', 'r') as f:
    with open('./input/server_responses_transfered_ebx', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx = pd.read_fwf('./input/server_responses_transfered_ebx', names=["Abfrage_ebx"])
df_result_neu = df_result.join(df_fwf_ebx)           #df_result_neu = pd.concat([df_result, df_fwf_ebx], axis=1)

In [35]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result_neu.shape[0]
y = df_fwf_ebx.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 2262 
Anzahl der Antworten vom Server:    2262


<hr>

### 2. Titel in Aleph vorhanden

= df_in_aleph

*Da erworbene Bücher nicht manuell gelöscht werden auch hier wieder Abgleich mit beiden Datenquellen.*

In [36]:
df_in_aleph["url_ges"] = df_in_aleph["isbn_ean"].apply(lambda x: f"ges_link{x}".replace('ges_link', 'http://aleph.mpg.de/X?op=find&base=ges01&request=IBS='))

df_in_aleph["url_ebx"] = df_in_aleph["isbn_ean"].apply(lambda x: f"ebx_link{x}".replace('ebx_link', 'http://aleph.mpg.de/X?op=find&base=ebx01&request=IBN='))


#### Abfragen beim Server


In [37]:
with open('./input/server_responses2', 'w') as fn:  
    for url in df_in_aleph["url_ges"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses2', 'r') as f:
    with open('./input/server_responses_transfered2', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ges = pd.read_fwf('./input/server_responses_transfered2', names=["Abfrage_ges"])
df_result2 = df_in_aleph.join(df_fwf_ges)               #df_result2 = pd.concat([df_in_aleph, df_fwf_ges], axis=1)             

In [38]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind, wird mittelfristig rausfallen
x = df_in_aleph.shape[0]
y = df_fwf_ges.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 18416 
Anzahl der Antworten vom Server:    18416


In [39]:
df_result2['Abfrage_ges']

0        neu
1        neu
2        neu
3        neu
4        neu
        ... 
18411    neu
18412    neu
18413    neu
18414    neu
18415    neu
Name: Abfrage_ges, Length: 18416, dtype: object

In [40]:
with open('./input/server_responses_ebx2', 'w') as fn:  
    for url in df_result2["url_ebx"]:
        reply = requests.get(url).text
        a = reply.replace('<?xml version = "1.0" encoding = "UTF-8"?>', '') 
        fn.write(a)

with open('./input/server_responses_ebx2', 'r') as f:
    with open('./input/server_responses_transfered_ebx2', 'w') as fr:
        for line in f:
            if 'empty' in line:
                fr.write('neu\n')
            elif 'no_records' in line:
                fr.write('vorhanden\n')

df_fwf_ebx2 = pd.read_fwf('./input/server_responses_transfered_ebx2', names=["Abfrage_ebx"])
df_result_in_aleph = df_result2.join(df_fwf_ebx2)           #df_result_in_aleph = pd.concat([df_result2, df_fwf_ebx2], axis=1)

In [41]:
#Kontrollabfrage, ob für alle Titel auch Treffer da sind
x = df_result_in_aleph.shape[0]
y = df_fwf_ebx2.shape[0]
print('Anzahl der eingelesenen Datensätze:', x, '\nAnzahl der Antworten vom Server:   ', y)

Anzahl der eingelesenen Datensätze: 18416 
Anzahl der Antworten vom Server:    18416


<hr>

## 3. Exportvorbereitungen 


### 1. Neue Titel

In [42]:
df_neu_nicht_einspielen =  df_result_neu.drop(df_result_neu[(df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu')].index)          
# alle Titel rausholen, die in einer der beiden Datenbanken vorhanden waren, diese werden mit den vorhandenen aus Update unten in Excel geschrieben


#### Jetzt Extraktion der Titel zum Einspielen:

In [43]:
df_neu_aleph_einspielen = df_result_neu.loc[((df_result_neu["Abfrage_ebx"]== 'neu') & (df_result_neu["Abfrage_ges"] == 'neu'))]   #das sind die komplett neuen Titel

### 2. In Aleph vorhandene Titel

*Hier ist der Fall: was im Ebooks-Katalog vorhanden ist, muss gelöscht werden*

In [44]:
df_in_aleph_nicht_einspielen = df_result_in_aleph.drop(df_result_in_aleph[(df_result_in_aleph["Abfrage_ebx"] == 'neu') & (df_result_in_aleph["Abfrage_ges"] == 'neu')].index) 
df_in_aleph_nicht_einspielen

,level_0,index,object_id,isbn_ean,title,subtitle,contributor,publisher,series,thesis,...,title_sep,subtitle_sep,subtitle_comparison,subtitle_comparison2,subtitle_all,short_title,url_ges,url_ebx,Abfrage_ges,Abfrage_ebx
4986,5459,7698,162619746,9780472133161,(Post-)colonial Archipelagos,Comparing the Legacies of Spanish Colonialism ...,Hans-Jurgen Burchardt;Johanna Leinius,The University of Michigan Press,,,...,(Post-)colonial Archipelagos,,Comparing the Legacies of Spanish Colonialism ...,,Comparing the Legacies of Spanish Colonialism ...,(Post-)colonial Archipelagos Comparing the Leg...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vor
6003,6582,9161,163232974,9781119890423,The Builder's Guide to the Tech Galaxy,99 Practices to Scale Startups into Unicorn Co...,Martin Schilling;Thomas Klugkist,Wiley,,,...,The Builder's Guide to the Tech Galaxy,,99 Practices to Scale Startups into Unicorn Co...,,99 Practices to Scale Startups into Unicorn Co...,The Builder's Guide to the Tech Galaxy 99 Prac...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vor
7313,7991,11084,163580891,9783593516363,Abschied von Kohle und Auto?,Sozial-ökologische Transformationskonflikte um...,"Dörre, Klaus",Campus,Labour Studies 26,,...,Abschied von Kohle und Auto?,,Sozial-ökologische Transformationskonflikte um...,,Sozial-ökologische Transformationskonflikte um...,Abschied von Kohle und Auto? Sozial-ökologisch...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
9454,10272,13986,163833446,9780192848901,Trade Wars,Past and Present,Nils Ole Oermann;Hans-Juergen Wolff,Oxford University Press,,,...,Trade Wars,,Past and Present,,Past and Present,Trade Wars Past and Present / Nils Ole Oermann...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
9600,10425,14211,161223477,9781260463255,Startup Program Design: A Practical Guide for ...,,Paolo Lombardi;Adam Berk,McGraw-Hill Education,,,...,Startup Program Design,A Practical Guide for Creating Accelerators a...,,A Practical Guide for Creating Accelerators a...,A Practical Guide for Creating Accelerators a...,Startup Program Design A Practical Guide for ...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,vor
10573,11469,15697,2576513,9781509553952,The Spirit of Digital Capitalism,,Huberman,Polity Press,,,...,The Spirit of Digital Capitalism,,,,,The Spirit of Digital Capitalism / Huberman,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
11498,12435,16901,163867562,9781032168197,Covid-19 and the Global Political Economy,Crises in the 21st Century,Tim Di Muzio,Taylor & Francis Ltd,RIPE Series in Global Political Economy,,...,Covid-19 and the Global Political Economy,,Crises in the 21st Century,,Crises in the 21st Century,Covid-19 and the Global Political Economy Cris...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
12804,13783,18781,164011468,9781803925776,A Research Agenda for Social Networks and Soci...,,Emmanuel Lazega,Edward Elgar Publishing,Elgar Research Agendas,,...,A Research Agenda for Social Networks and Soci...,,,,,A Research Agenda for Social Networks and Soci...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
12805,13784,18782,164584573,9781800378797,A Research Agenda for Intelligence Studies and...,,Robert Dover,Edward Elgar Publishing,Elgar Research Agendas,,...,A Research Agenda for Intelligence Studies and...,,,,,A Research Agenda for Intelligence Studies and...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,vor,neu
12807,13786,18784,164487443,9781800889194,Assessing the Performance Advantage of

In [45]:
# Erzeugen der Datei zum Löschen der Titel

with open("./output/ges02_weg", "a", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_in_aleph_nicht_einspielen.index:
        fa.write(df_in_aleph_nicht_einspielen["ids"][i]+'GES02'+'\n')

Für die Statistik und Kontrolle zusammenführen aller Titel, die nicht eingespielt werden und Ausgabe in einer CSV

In [46]:
df_gesamt_nicht_einspielen = df_in_aleph_nicht_einspielen.append(df_neu_nicht_einspielen)   

In [47]:
date = time.strftime("%Y_%m_%d")                                              # Zeit erfassen für Dateibenennung

df_gesamt_nicht_einspielen["object_id"] = df_gesamt_nicht_einspielen.object_id.astype(str)  # wandelt die spalte von Int64 zu Object um, so dass es in Excel korrekt eingelesen wird
df_gesamt_nicht_einspielen["isbn_ean"] = df_gesamt_nicht_einspielen.isbn_ean.astype(str)
df_gesamt_nicht_einspielen = df_gesamt_nicht_einspielen.drop(columns=["url_ebx", "url_ges", "cover", "title_sep", "subtitle_comparison", "subtitle_comparison2", "subtitle_all", "subtitle_sep"]) # unnötige Spalten entfernen

df_neu_nicht_einspielen.to_csv('./output/Vorhandene_Titel_'+date+'.csv')   

#### Jetzt Extraktion der Titel zum Updaten:

In [48]:
df_in_aleph_update = df_result_in_aleph.loc[(df_result_in_aleph["Abfrage_ebx"]== 'neu') & (df_result_in_aleph["Abfrage_ges"] == 'neu')] # prüfen, ob es wirklich ein Datenupdate gab, sonst nicht neu einspielen??

<hr>

#### Für die Logdatei Ermittlung verschiedener Zahlen und hier zur direkten Ansicht ausgegeben

In [49]:
#Kontrollmechanismus, ob für alle Titel auch Treffer da sind
x = df.shape[0]
a = df_ohne_dubletten.shape[0]
c = df_dubletten.shape[0]
b = df_dubl_einspielen.shape[0] #Auswahl der neuen Treffer
m = df_in_aleph_nicht_einspielen.shape[0]
n = df_ganz_neu.shape[0]
h = df_neu_aleph_einspielen.shape[0]
j = df_neu_nicht_einspielen.shape[0]
o = df_in_aleph_update.shape[0]
g = df_in_aleph.shape[0]
k = df_alephIDs.shape[0]
z = df_neu_aleph_einspielen.shape[0]       #neu ermitteln aus neuen
l = df_aleph_loeschen.shape[0]
y = df_dubl_nicht_einspielen.shape[0]


print('Kleine Statistik:\n=====================================',
    '\nGelieferte Datensätze:             ', x,
    '\n--------------------------------------------', 
    '\nSätze ohne Dubletten               ', a,
    '\n   Dubletten:              ', c,
    '\n   Auswahl zum Einspielen  ', b,
    '\n   in Aleph löschen        ', y,
    '\nZu prüfende Datensätze:            ', a+b,
    '\n--------------------------------------------', 
    '\nPrüfung Titel in Aleph:            ', g,
    '\n   Davon als Update        ', o,
    '\n   Davon in Aleph löschen  ', m ,    
    '\nPrüfung neue Titel    :            ', n,
    '\n   Davon ganz neu          ', h,
    '\n   Davon bereits Bestand   ', j,      
    '\n--------------------------------------------', 
    '\nTitel, die in Aleph verarbeitet werden: ', o+h,
    '\n--------------------------------------------', 
    '\nDatensätze GES02 vor Einspielen:   ', k,
    '\nNicht mehr im Export               -', l,
    '\nTitel in Aleph löschen, da Bestand -', m,
    '\nZu löschen, da Dublette            -', y,
    '\nNeue Titel für Aleph               +', h,
    '\n--------------------------------------------', 
    '\nIn Aleph nach Einspielen:          ', k-l-m-y+h,)

#hier entsprechende Einträge für die Log-Datei

with open ('./log/pda_import_log.txt', 'a') as log:
    log.write("\nGelieferte Datensätze:             " + str(x))
    log.write("\n--------------------------------------------\n")
    log.write("\nSätze ohne Dubletten               " + str(a))
    log.write("\n   Dubletten:              " + str(c))
    log.write("\n   Davon in Aleph löeschen " + str(y))
    log.write("\n   Auswahl zum Einspielen  " + str(b))
    log.write("\nZu prüfende Datensätze:            " + str(a+b))
    log.write("\n--------------------------------------------\n")
    log.write("\nPrüfung Titel in Aleph:            " + str(g))
    log.write("\n   Davon als Update        " + str(o))
    log.write("\n   Davon in Aleph löschen  " + str(m) + "    \n")   
    log.write("\nPrüfung neue Titel    :            " + str(n))
    log.write("\n   Davon ganz neu          " + str(h))
    log.write("\n   Davon bereits Bestand   " + str(j) + "    \n")     
    log.write("\n--------------------------------------------\n")
    log.write("\nTitel, die in Aleph verarbeitet werden: " + str(o+h))
    log.write("\n--------------------------------------------\n")
    log.write("\nDatensätze GES02 vor Einspielen:   " + str(k))
    log.write("\nNicht mehr im Export               -" + str(l))
    log.write("\nZu löschender dubletter Titel      -" + str(y))
    log.write("\nTitel in Aleph löschen, da Bestand -" + str(m))
    log.write("\nNeue Titel für Aleph               +" + str(h))
    log.write("\n--------------------------------------------\n")
    log.write("\nIn Aleph nach Einspielen:          " + str(k-l-m-y+h))
    log.write("\n--------------------------------------------\n")

Kleine Statistik:
Gelieferte Datensätze:              23894 
-------------------------------------------- 
Sätze ohne Dubletten                17550 
   Dubletten:               6344 
   Auswahl zum Einspielen   3128 
   in Aleph löschen         0 
Zu prüfende Datensätze:             20678 
-------------------------------------------- 
Prüfung Titel in Aleph:             18416 
   Davon als Update         18380 
   Davon in Aleph löschen   36 
Prüfung neue Titel    :             2262 
   Davon ganz neu           1072 
   Davon bereits Bestand    1190 
-------------------------------------------- 
Titel, die in Aleph verarbeitet werden:  19452 
-------------------------------------------- 
Datensätze GES02 vor Einspielen:    20651 
Nicht mehr im Export               - 57 
Titel in Aleph löschen, da Bestand - 36 
Zu löschen, da Dublette            - 0 
Neue Titel für Aleph               + 1072 
-------------------------------------------- 
In Aleph nach Einspielen:           21630


<hr>

## 4. Exportdateien Aufbereiten

#### Zielformat für das Einspielen in Aleph:

    000000001 LDR   L -----nM2.01200024------h              
    000000001 020   L $$a (object_id))
    000000001 030   L $$aaz||rrrza||||
    000000001 051   L $$am|||||||
    000000001 070   L $$aSchweitzer
    000000001 077   L $$aMonographie
    000000001 078   L $$aSchweitzer
    000000001 082   L $$azum Bestellen
    000000001 100   L $$a (contributor_1)
    000000001 104   L $$a (contributor_2)
    000000001 108   L $$a (contributor_3)
    000000001 331   L $$a (title_sep)
    000000001 335   L $$a (subtitle_all)
    000000001 403   L $$a (edition_number / edition_text)  #noch prüfen, was besser zu verwenden ist 
    000000001 419   L $$b (publisher) $$a (date_combined)
    000000001 433   L $$a (pages)
    000000001 451   L $$a (series)
    000000001 520   L $$a (thesis)
    000000001 540   L $$a (isbn_ean)
    000000001 656   L $$a (cover)
    000000001 750   L $$a (description)
    000000001 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch        
    
Anmerkung zum Feld 655: die URL wird NACH dem Einspielen in Aleph mit der Datensatz-ID angereichert (siehe Juypter-Notebook "Link-Anreicherung"), um einen klaren Bestellink für den Kaufvorschlag zu haben

*Hierfür werden immer die Feldbenennung und bestimmte Codierungen VOR den Inhalt - in Klammern de Bezeichnung der entsprechenden Spalte - gesetzt, bzw. erfoderliche Felder komplett neu hinzugefügt.   
Am Anfang jeder Zeile braucht Aleph eine 9-Stellige eindeutige Zahl pro Titel.*   

*Manchmal ließ sich der Inhalt einer Spalte direkt in die Datei schreiben, manchmal musst die Spalte zuvor über apply aufbereitet werden.* 

## 1. Aufbereiten der neuen Titel

In [50]:
df_aleph_einspielen = df_neu_aleph_einspielen           #zur vereinfachten Wiederverwertung des alten Codes

In [51]:
df_aleph_einspielen["020"] = df_aleph_einspielen["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_aleph_einspielen["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

#### Besondere Aufbereitung der Personendaten

*Da bis zu 3 Personen in einer Spalte zu finden sind, werden diese im Discovery nicht getrennt suchbar, darum werden sie gesplittet. Für die Dublettenkontrolle hat sich das als irrelevant erwiesen, darum erfolgt dieser Schritt erst hier.*

In [52]:
person = df_aleph_einspielen["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_aleph_einspielen["contributor_1"]= person[0]
df_aleph_einspielen["contributor_2"]= person[1]
df_aleph_einspielen["contributor_3"]= person[2]

df_aleph_einspielen["contributor_1"]= df_aleph_einspielen["contributor_1"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_2"]= df_aleph_einspielen["contributor_2"].replace(np.nan, '', regex=True)
df_aleph_einspielen["contributor_3"]= df_aleph_einspielen["contributor_3"].replace(np.nan, '', regex=True)


#### Besondere Aufbereitung des Erscheinungsdatum und Erscheinungsjahres

*In der Auswahl unserer Titel befinden sich auch im Erscheinen befindliche Titel der kommenden Monate. Diese Information möchten wir gerne im Discovery sichtbar machen. Hierfür bleibt uns nur Aleph-Feld 419c, das dem Erscheinungsjahr vorbehalten ist.   
Wunsch ist es: Wenn des Erscheinungsdatum weiter als 10 Tage weg vom heutigen Datum ist, soll das komplette Datum angezeigt werden, ansonsten nur das Erscheinungsjahr.*

Zur Umsetzung muss die Spalte "publication_date" in ein Datum verwandelt werden und nach den genannten Kriterien unterschiedlich angezeigt

In [53]:
today = int(time.strftime('%Y%m%d'))
today

20221221

# Test weil DAtuem hakt --

In [54]:
df_aleph_einspielen ["publication_DATUM"] = df_aleph_einspielen["publication_date"].apply(pd.to_numeric,errors='ignore')



In [55]:
# df_aleph_einspielen ["publication_DATUM"].unique()

df_aleph_einspielen["publication_date2"] = df_aleph_einspielen["publication_date"].astype(float)

df_aleph_einspielen["publication_date_year"] = df_aleph_einspielen["publication_date"].astype(str).str.slice(start=0,stop=4)    #einfaches Zerlegen in die Datumsbestandteile und anschließendes Zusammenfügen
#df_aleph_einspielen["publication_date_month"] = df_aleph_einspielen["publication_date"].astype(str).str.slice(start=4,stop=6)
#df_aleph_einspielen["publication_date_day"] = df_aleph_einspielen["publication_date"].astype(str).str.slice(start=6,stop=8)

df_aleph_einspielen["publication_date_year"]

df_aleph_einspielen['publicatin_date_date'] = pd.to_datetime(df_aleph_einspielen.publication_date)          #Umwandlung, da Spalteninhalt object ist
df_aleph_einspielen['publicatin_date_date']

df_aleph_einspielen["publication_date_korr"] = df_aleph_einspielen["publication_date"].astype(str).str.replace('00','01')
df_aleph_einspielen['publicatin_date_date'] = df_aleph_einspielen["publication_date_korr"].astype(int)          #Umwandlung, da Spalteninhalt object ist
df_aleph_einspielen['publicatin_date_date']

In [56]:
df_aleph_einspielen["coming_soon"] = np.where(df_aleph_einspielen["publication_DATUM"] > today+10, df_aleph_einspielen["publication_DATUM"], np.nan)

In [57]:
df_aleph_einspielen["coming_soon"] 

88             NaN
227            NaN
228            NaN
240            NaN
274            NaN
           ...    
2255    20230201.0
2257           NaN
2258           NaN
2260           NaN
2261           NaN
Name: coming_soon, Length: 1072, dtype: float64

# Ende test

#today = int(time.strftime('%Y%m%d'))
df_aleph_einspielen["coming_soon"] = np.where(df_aleph_einspielen["publication_date"].astype(int) > today+10, df_aleph_einspielen["publication_date"], np.nan) #zieht die über 10 Tage raus, brauchen nan für Umwandlung in Datum

In [58]:
df_aleph_einspielen["publication_date_soon"] = df_aleph_einspielen["coming_soon"].astype(str).str.replace('00','01')
#df_aleph_einspielen["publication_date_soon"]

In [59]:
df_aleph_einspielen["year"] = df_aleph_einspielen["publication_date_soon"].astype(str).str.slice(start=0,stop=4)    #einfaches Zerlegen in die Datumsbestandteile und anschließendes Zusammenfügen
df_aleph_einspielen["month"] = df_aleph_einspielen["publication_date_soon"].astype(str).str.slice(start=4,stop=6)
df_aleph_einspielen["day"] = df_aleph_einspielen["publication_date_soon"].astype(str).str.slice(start=6,stop=8)

In [60]:
df_aleph_einspielen["full_coming_soon"] = df_aleph_einspielen["year"]+'-'+df_aleph_einspielen["month"]+'-'+df_aleph_einspielen["day"]
df_aleph_einspielen["coming_soon"] = df_aleph_einspielen["full_coming_soon"].astype(str).str.replace('nan--','') 

In [61]:
df_aleph_einspielen["coming_soon"] 

88                
227               
228               
240               
274               
           ...    
2255    2023-02-01
2257              
2258              
2260              
2261              
Name: coming_soon, Length: 1072, dtype: object

In [62]:
df_aleph_einspielen["published"] = np.where(df_aleph_einspielen["coming_soon"] == '', df_aleph_einspielen["publication_year"], '') #Auslesen und Kombinieren der Daten
df_aleph_einspielen["year_publ"] = df_aleph_einspielen["published"].astype(str).str.slice(start=0,stop=4)                          #da wieder .0 am Ende war, Jahreszahl ausschneidens
df_aleph_einspielen["date_combined"] = df_aleph_einspielen["year_publ"]+df_aleph_einspielen["coming_soon"]
df_aleph_einspielen['date_combined']

88            2021
227           2021
228           2021
240           2022
274           2021
           ...    
2255    2023-02-01
2257          2022
2258          2022
2260          2021
2261          2022
Name: date_combined, Length: 1072, dtype: object

In [63]:
#df_aleph_einspielen.info()

In [64]:
df_aleph_einspielen['isbn_ean'] = df_aleph_einspielen['isbn_ean'].astype(np.int64)   #da die Zahl als Object genommen wurde, bekam sie ein .0 angehängt, das ist durch umwandeln in Zahl weg

df_aleph_einspielen['isbn_ean']

88      9780197532355
227     9780190063054
228     9780197583180
240     9780197555972
274     9780197567654
            ...      
2255    9780774866361
2257    9780520374010
2258    9781350026995
2260    9781538151358
2261    9780367426569
Name: isbn_ean, Length: 1072, dtype: int64

In [65]:
df_aleph_einspielen["419b"] = df_aleph_einspielen["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_aleph_einspielen["419c"] = df_aleph_einspielen["date_combined"].apply(lambda x: f"$$c{x}")                  

df_aleph_einspielen["419"] = df_aleph_einspielen["419b"]+df_aleph_einspielen["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

In [66]:
df_aleph_einspielen["403"] = df_aleph_einspielen["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_aleph_einspielen["433"] = df_aleph_einspielen["pages"].apply(lambda x: f"433   L $$b{x}")
df_aleph_einspielen["451"] = df_aleph_einspielen["series"].apply(lambda x: f"451   L $$b{x}") 
df_aleph_einspielen["520"] = df_aleph_einspielen["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_aleph_einspielen["540"] = df_aleph_einspielen["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_aleph_einspielen["656"] = df_aleph_einspielen["cover"].apply(lambda x: f"656   L $$u{x}") 
df_aleph_einspielen["750"] = df_aleph_einspielen["description"].apply(lambda x: f"750   L $$a{x}") 

#### Für das Durchzählen der Titel braucht es eine neue Spalte

In [67]:
# hier entsteht eine neue Spalte mit Zahlen ab 1 durchgehend gezählt, die für den korrekten Import der Daten in Aleph nötig ist
x = df_aleph_einspielen.shape[0]   
df_aleph_einspielen["id"] = range(1,x+1)                                                       #Notwendig ist die Zählung ab 1, da Aleph sonst nicht korrekt einließt
df_aleph_einspielen["id"] = df_aleph_einspielen["id"].apply(lambda x: f"{x:09d}")              #Die Zahl muss 9-Stellig aufgefüllt werden



#### Vorbereitungen abgeschlossen, jetzt das Schreiben der Datei im Aleph-Sequential-Format:

In [68]:
with open("./output/ges02_neu", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_aleph_einspielen.index:
        fa.write(df_aleph_einspielen["id"][i]+' LDR   L -----nM2.01200024------h'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["020"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 030   L $$aaz||rrrza||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 051   L $$am|||||||m|||||||'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 077   L $$aMonographie'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 078   L $$aSchweitzer'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 082   L $$azum Bestellen'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 100   L $$a'+df_aleph_einspielen["contributor_1"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 104   L $$a'+df_aleph_einspielen["contributor_2"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 108   L $$a'+df_aleph_einspielen["contributor_3"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 331   L $$a'+df_aleph_einspielen["title_sep"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 335   L $$a'+df_aleph_einspielen["subtitle_all"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["403"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["419"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["433"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["451"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["520"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["540"][i]+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' 655   L $$zOrder me$$umailto:bib@mpifg.de?subject=Bestellwunsch'+'\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["656"][i]+'$$3Cover\n')
        fa.write(df_aleph_einspielen["id"][i]+' '+df_aleph_einspielen["750"][i]+'\n')

<hr>

### 2. Daten für Update

*es wird anhand der Spalte "last_modified" geprüft, ob die Titel seit dem letzten Einspielen ein Update erfahren haben*


 Um die Menge zum Updaten zu reduzieren, werden nur die rausgezogen, die tatsächlich ein Aktualisierungsdatum haben

In [69]:
tday = datetime.datetime.now()
td = datetime.timedelta(days = 7)
u = tday - td
df_in_aleph_update['timespan'] = u                                                           # Einfügen einer Spalte mit Datum vor 10 Tagen als Basis für Abfrage zu Update-Notwendigkeit
print(u)

2022-12-14 11:35:28.070985


In [70]:
df_in_aleph_update['last_modified'] = pd.to_datetime(df_in_aleph_update.last_modified)          #Umwandlung, da Spalteninhalt object ist
df_in_aleph_update['last_modified']

0       2022-12-19 18:18:36
1       2022-12-20 18:13:11
2       2022-12-16 17:20:09
3       2022-11-25 18:47:48
4       2022-12-20 17:40:29
                ...        
18411   2022-12-16 17:22:50
18412   2022-12-01 17:18:31
18413   2022-12-19 17:58:30
18414   2022-12-16 18:05:12
18415   2022-12-16 18:20:42
Name: last_modified, Length: 18380, dtype: datetime64[ns]

In [71]:
df_in_aleph_update['true'] = np.where(df_in_aleph_update['last_modified'] > u, df_in_aleph_update['last_modified'], np.datetime64('NaT'))
df_in_aleph_update['true']

0       2022-12-19 18:18:36
1       2022-12-20 18:13:11
2       2022-12-16 17:20:09
3                       NaT
4       2022-12-20 17:40:29
                ...        
18411   2022-12-16 17:22:50
18412                   NaT
18413   2022-12-19 17:58:30
18414   2022-12-16 18:05:12
18415   2022-12-16 18:20:42
Name: true, Length: 18380, dtype: datetime64[ns]

In [72]:
df_in_aleph_update_true = df_in_aleph_update[pd.notnull(df_in_aleph_update['true'])]

In [73]:
df_in_aleph_update_true

,level_0,index,object_id,isbn_ean,title,subtitle,contributor,publisher,series,thesis,...,subtitle_comparison,subtitle_comparison2,subtitle_all,short_title,url_ges,url_ebx,Abfrage_ges,Abfrage_ebx,timespan,true
0,0,0,160055285,9783404070022,Fake Facts,Wie Verschwörungstheorien unser Denken bestimmen,Katharina Nocun;Pia Lamberty,Quadriga,,,...,Wie Verschwörungstheorien unser Denken bestimmen,,Wie Verschwörungstheorien unser Denken bestimmen,Fake Facts Wie Verschwörungstheorien unser Den...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,neu,2022-12-14 11:35:28.070985,2022-12-19 18:18:36
1,1,5,161274400,9783949098031,Der Preis der Zukunft,Warum Deflation der Schlüssel zum Wohlstand vo...,Jeff Booth,Aprycot Media,,,...,Warum Deflation der Schlüssel zum Wohlstand vo...,,Warum Deflation der Schlüssel zum Wohlstand vo...,Der Preis der Zukunft Warum Deflation der Schl...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,neu,2022-12-14 11:35:28.070985,2022-12-20 18:13:11
2,2,7,158945668,9780231550734,Experiencing Design,The Innovator's Journey,Jeanne Liedtka;Karen Hold;Jessica Eldridge,Columbia Business School Publishing,,,...,The Innovator's Journey,,The Innovator's Journey,Experiencing Design The Innovator's Journey / ...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,neu,2022-12-14 11:35:28.070985,2022-12-16 17:20:09
4,6,20,156899389,9781529110630,Another Now,Dispatches from an Alternative Present from th...,Yanis Varoufakis,Vintage,,,...,Dispatches from an Alternative Present from th...,,Dispatches from an Alternative Present from th...,Another Now Dispatches from an Alternative Pre...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,neu,2022-12-14 11:35:28.070985,2022-12-20 17:40:29
8,11,28,161205208,9783763961061,Qualitätsoffensive Diversität,Nachwuchsgewinnung und -bildung in verschieden...,Ulrike Senger,wbv Publikation,Promovieren neu gestalten,,...,Nachwuchsgewinnung und -bildung in verschieden...,,Nachwuchsgewinnung und -bildung in verschieden...,Qualitätsoffensive Diversität Nachwuchsgewinnu...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,neu,2022-12-14 11:35:28.070985,2022-12-16 05:53:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18406,20667,15749,160887141,9781108736374,A Concise History of Canada,,Margaret Conrad,Cambridge University Press,Cambridge Concise Histories,,...,,,,A Concise History of Canada / Margaret Conrad,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,neu,2022-12-14 11:35:28.070985,2022-12-18 17:46:18
18411,20672,12985,162877975,9780367490157,A Beginner's Guide to Structural Equation Mode...,,Tiffany A. Whittaker;Randall E. Schumacker,Routledge,,,...,,,,A Beginner's Guide to Structural Equation Mode...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,neu,2022-12-14 11:35:28.070985,2022-12-16 17:22:50
18413,20675,20675,163840881,9781487555450,1950s Canada,Politics and Public Affairs,Nelson Wiseman,University of Toronto Press,,,...,Politics and Public Affairs,,Politics and Public Affairs,1950s Canada Politics and Public Affairs / Nel...,http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque...,neu,neu,2022-12-14 11:35:28.070985,2022-12-19 17:58:30
18414,20676,22052,164644335,9781982164027,"100,000 First Bosses",My Unlikely Path as a 22-Year-Old Lawmaker,Will Haskell,Avid Reader Press / Simon & Schuster,,,...,My Unlikely Path as a 22-Year-Old Lawmaker,,My Unlikely Path as a 22-Year-Old Lawmaker,"100,000 First Bosses My Unlikely Path as a 22-...",http://aleph.mpg.de/X?op=find&base=ges01&reque...,http://aleph.mpg.de/X?op=find&base=ebx01&reque

#### Erstellen der Update-Export-Datei

In [74]:
#Zwischenschritt, um für alle in Aleph das Update zu starten
#df_in_aleph_update_true = df_in_aleph_update

In [75]:
df_in_aleph_update_true["020"] = df_in_aleph_update_true["object_id"].apply(lambda x: f"020   L $$a{x}") 
del df_in_aleph_update_true["object_id"]                                                                      #um das df nicht unnötig anwachsen zu lassen, jeweils alte Spalte löschen

In [76]:
#Aufbereitung Personendaten

person = df_in_aleph_update_true["contributor"].str.split(';', expand=True)                         #Für saubere Daten, die Autorenangabe splitten und in getrennte Felder schreiben

df_in_aleph_update_true["contributor_1"]= person[0]
df_in_aleph_update_true["contributor_2"]= person[1]
df_in_aleph_update_true["contributor_3"]= person[2]

df_in_aleph_update_true["contributor_1"]= df_in_aleph_update_true["contributor_1"].replace(np.nan, '', regex=True)
df_in_aleph_update_true["contributor_2"]= df_in_aleph_update_true["contributor_2"].replace(np.nan, '', regex=True)
df_in_aleph_update_true["contributor_3"]= df_in_aleph_update_true["contributor_3"].replace(np.nan, '', regex=True)


### Neue Schritte zur Aufbereitung des Erscheinungsdatums, da der Code von oben hier nicht ging - warum auch immer

In [77]:
#today = int(time.strftime('%Y%m%d'))
#soon = today+10

In [78]:
df_in_aleph_update_true['publ_date_repl'] = df_in_aleph_update_true['publication_date'].astype(str)

In [79]:
df_in_aleph_update_true['publ_date_repl']

0        20210827
1        20210800
2        20210720
4        20210909
8        20210701
           ...   
18406    20220811
18411    20220506
18413    20221109
18414    20230110
18415    20221124
Name: publ_date_repl, Length: 6936, dtype: object

In [80]:
df_in_aleph_update_true['publ_date_repl'] = df_in_aleph_update_true['publ_date_repl'].str.replace('00','01')

In [81]:
df_in_aleph_update_true['publ_date_repl'].value_counts()

20220801    106
20221215     83
20220819     78
20221201     71
20221230     69
           ... 
20211128      1
20150313      1
20150217      1
20220529      1
20201921      1
Name: publ_date_repl, Length: 768, dtype: int64

In [82]:
df_in_aleph_update_true['publ_date_repl_date'] = pd.to_datetime(df_in_aleph_update_true['publ_date_repl'], errors='coerce')
df_in_aleph_update_true['publ_date_repl_date']

0       2021-08-27
1       2021-08-01
2       2021-07-20
4       2021-09-09
8       2021-07-01
           ...    
18406   2022-08-11
18411   2022-05-06
18413   2022-11-09
18414   2023-01-10
18415   2022-11-24
Name: publ_date_repl_date, Length: 6936, dtype: datetime64[ns]

In [83]:
df_in_aleph_update_true["coming_soon"] = np.where(df_in_aleph_update_true["publ_date_repl_date"] > u, df_in_aleph_update_true["publ_date_repl_date"], np.datetime64('NaT'))  #df_in_aleph_update_true['today'])

In [84]:
df_in_aleph_update_true["coming_soon"]

0              NaT
1              NaT
2              NaT
4              NaT
8              NaT
           ...    
18406          NaT
18411          NaT
18413          NaT
18414   2023-01-10
18415          NaT
Name: coming_soon, Length: 6936, dtype: datetime64[ns]

In [85]:
#df_in_aleph_update_true["coming_soon"].replace('NaT','')

In [86]:
df_in_aleph_update_true["date_combined"] = np.where(df_in_aleph_update_true["coming_soon"].astype(str) == 'NaT', df_in_aleph_update_true["publication_year"], df_in_aleph_update_true["coming_soon"].astype(str))
df_in_aleph_update_true["date_combined"]


0              2021
1              2021
2              2021
4              2021
8              2021
            ...    
18406          2022
18411          2022
18413          2022
18414    2023-01-10
18415          2022
Name: date_combined, Length: 6936, dtype: object

In [87]:
df_in_aleph_update_true['isbn_ean'] = df_in_aleph_update_true['isbn_ean'].astype(np.int64)   #da die Zahl als Object genommen wurde, bekam sie ein .0 angehängt, das ist durch umwandeln in Zahl weg

df_in_aleph_update_true['isbn_ean']

0        9783404070022
1        9783949098031
2        9780231550734
4        9781529110630
8        9783763961061
             ...      
18406    9781108736374
18411    9780367490157
18413    9781487555450
18414    9781982164027
18415    9783030875183
Name: isbn_ean, Length: 6936, dtype: int64

In [88]:
# Bearbeitung der Felder

df_in_aleph_update_true["419b"] = df_in_aleph_update_true["publisher"].apply(lambda x: f"419   L $$b{x}") 
df_in_aleph_update_true["419c"] = df_in_aleph_update_true["date_combined"].apply(lambda x: f"$$c{x}")                  

df_in_aleph_update_true["419"] = df_in_aleph_update_true["419b"]+df_in_aleph_update_true["419c"]                           #Für die Korrekte Eingabe brauche ich Verlag und Jahr in einer Spalte

df_in_aleph_update_true["403"] = df_in_aleph_update_true["edition_text"].apply(lambda x: f"403   L $$a{x}") 
df_in_aleph_update_true["433"] = df_in_aleph_update_true["pages"].apply(lambda x: f"433   L $$b{x}")
df_in_aleph_update_true["451"] = df_in_aleph_update_true["series"].apply(lambda x: f"451   L $$b{x}") 
df_in_aleph_update_true["520"] = df_in_aleph_update_true["thesis"].apply(lambda x: f"520   L $$a{x}") 
df_in_aleph_update_true["540"] = df_in_aleph_update_true["isbn_ean"].apply(lambda x: f"540   L $$a{x}") 
df_in_aleph_update_true["656"] = df_in_aleph_update_true["cover"].apply(lambda x: f"656   L $$u{x}") 
df_in_aleph_update_true["750"] = df_in_aleph_update_true["description"].apply(lambda x: f"750   L $$a{x}") 

In [89]:
# Schreiben der Ausgabedatei, hier kleine Unterschiede zu den neuen Titeln. Vorhandene ids und bestimmte Felder können nicht verändert sein, brauchen also nicht übernommen zu werden.

with open("./output/ges02_update", "w", encoding="utf-8") as fa:  #durch das Encoding hier, kommen Sonderzeichen richtig rüber
    for i in df_in_aleph_update_true.index:
        fa.write(df_in_aleph_update_true["ids"][i]+' 100   L $$a'+df_in_aleph_update_true["contributor_1"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 104   L $$a'+df_in_aleph_update_true["contributor_2"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 108   L $$a'+df_in_aleph_update_true["contributor_3"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 331   L $$a'+df_in_aleph_update_true["title_sep"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' 335   L $$a'+df_in_aleph_update_true["subtitle_all"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["403"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["419"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["433"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["451"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["520"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["540"][i]+'\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["656"][i]+'$$3Cover\n')
        fa.write(df_in_aleph_update_true["ids"][i]+' '+df_in_aleph_update_true["750"][i]+'\n')

<hr>

## Abschließende Dinge

In [90]:
#Abschließender Eintrag in Log-Datei
endtime = time.strftime('%H:%M')

with open ('./log/pda_import_log.txt', 'a') as log:                                                  # Da diese Log-Datei nicht unmittelbar gebraucht wird, hier fortlaufendes Schreiben in eine Datei
    log.write('\n                                     beendet ')
    log.write(endtime)
    log.write("\n============================================================\n\n")

Kopien bestimmter Daten zur Einsicht bzw. für Prüfzwecke

In [91]:
df_aleph_einspielen.to_csv('./output/Eingespielte_Titel_'+date+'.csv') 
df_in_aleph_update_true.to_csv('./output/Update_Titel_'+date+'.csv')

df_in_aleph_nicht_einspielen.to_csv('./output/Aleph_loeschen_ebx_vorh_'+date+'.csv')

<hr>

Folgende Schritte müssen mit den Dateien ausgeführt werden: 


1. Einspielen der Datei pda_ges01 als neue Titel in Aleph, hier dann auch Export der urls und Anreicherung mit der Aleph-ID mittels "mailto_link_skript.ipynb"
2. Einspielen der DAtei pda_update als "Änderungen bestehender Datensätze in Aleph"
3. Einspielen und löschen der Titel die in ges02_loeschen_1 und ges02_loeschen_2 vorhanden sind